In [4]:
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import backend as K

from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

import matplotlib.pyplot as plt

import numpy as np


from tensorflow.keras.applications.vgg16 import VGG16 as Model

model =load_model('models/vgg_16_10e.h5')

replace2linear = ReplaceToLinear()


img1 = load_img('NCT-CRC-HE-small/BACK/BACK-AAHIIGKS.tif.jpg', target_size=(224, 224))
img2 = load_img('NCT-CRC-HE-small/DEB/DEB-AAGWLDAI.tif.jpg', target_size=(224, 224))
img3 = load_img('NCT-CRC-HE-small/LYM/LYM-ADHLDHYC.tif.jpg', target_size=(224, 224))

images = np.asarray([np.array(img1), np.array(img2), np.array(img3)])
image_titles = ['BACK', 'DEB', 'LYM']

score = CategoricalScore([1, 2, 3])


# Create Gradcam object
gradcam = Gradcam(model,
                  model_modifier=replace2linear,
                  clone=True)

X = preprocess_input(images)

# Generate heatmap with GradCAM
cam = gradcam(score,
              X,
              penultimate_layer=-1)

## Since v0.6.0, calling `normalize()` is NOT necessary.
# cam = normalize(cam)

# Render
f, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
for i, title in enumerate(image_titles):
    heatmap = np.uint8(cm.jet(cam[i])[..., :3] * 255)
    ax[i].set_title(title, fontsize=16)
    ax[i].imshow(images[i])
    ax[i].imshow(heatmap, cmap='jet', alpha=0.5) # overlay
    ax[i].axis('off')
plt.tight_layout()
plt.show()

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_5'), name='input_5', description="created by layer 'input_5'") at layer "block1_conv1". The following previous layers were accessed without issue: []